In [1]:
from azureml.core import Workspace, Experiment, Run, Datastore, Dataset, Environment

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.44.0 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('azureml-core~=1.43.0'), {'azureml-telemetry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-core 1.44.0 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('azureml-core~=1.43.0'), {'azureml-train-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.44.0 (c:\users\satya\anaconda3\envs\tf-gpu-cuda8\lib\site-packages), Requirement.parse('azureml-core~=1.43.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun =

In [3]:
#used my own azure pay as go account.

#workspace=Workspace.create(name='azuremlsdk-ws01',subscription_id='1db33695-8135-4616-9bb4-9574b401d454',
#resource_group='azuremlsdk-rg01',create_resource_group=True,
#location='eastus2')

workspace=Workspace.from_config('config.json')

In [16]:
#registering the external datastore
Datastore_Register=Datastore.register_azure_blob_container(workspace=workspace, datastore_name='loandatastore',account_key='Bb9kCARVNR0gHaNQqzaMLh3WC7HBrcom8ZjbSuuo8xcIQxYuXWF5isokFvsQfNwmOTlhdYcSkJY9+AStbu5jeg==',
container_name='loans',account_name='storagerg011',create_if_not_exists=True)


In [4]:
Datastore_retriveal=Datastore.get(workspace,'loandatastore')

In [31]:
#forming directories for the container
csv=[(Datastore_retriveal,'Loan_Default.csv')]
dataset1=Dataset.Tabular.from_delimited_files(path=csv)
dataset=dataset1.register(workspace,'loansdataset',create_new_version=True)
#conveting the azure dataset to pandas dataframe
df=dataset.to_pandas_dataframe()

In [6]:
from azureml.core import conda_dependencies, ScriptRunConfig

#sanity checks and script setup # Library imports for the virtual machine
experiment=Experiment(workspace,'loanexperiment')
environment=Environment('loanenv')
dependencies=['pandas','scikit-learn']
mydep=conda_dependencies.CondaDependencies.create(conda_packages=dependencies)
environment.python.conda_dependencies=mydep
environment.register(workspace=workspace)
#importing computes
from azureml.core.compute import AmlCompute
compute_name="cluster"
compute_config=AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=2,)
compute=AmlCompute.create(workspace=workspace,name=compute_name,provisioning_configuration=compute_config)
compute.wait_for_completion(show_output=True)






InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


NameError: name 'azureml' is not defined

In [16]:
#script
from azureml.core import ScriptRunConfig
scriptconfig=ScriptRunConfig(source_directory='.', script='train_test_SVC.py', environment=environment, compute_target=compute, arguments=['--input-data',dataset.as_named_input('raw_data')])
#hyperdrive
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal, choice
param_sampling=RandomParameterSampling(
{'C':choice(0.1,2,10,100),
'kernel':choice('linear','rbf','poly'),
'degree':choice(2,3,4,5,6,7,8,9,10)
}) 
hyperdrive_run=HyperDriveConfig(run_config=scriptconfig,hyperparameter_sampling=param_sampling,primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=20,
max_concurrent_runs=7,policy=BanditPolicy(evaluation_interval=2,slack_factor=0.1))
new_run=experiment.submit(hyperdrive_run,snapshot_directory=None)
new_run.wait_for_completion(show_output=True)


RunId: HD_1cf58c0b-ece7-4f71-8081-fdee5b6fd06d
Web View: https://ml.azure.com/runs/HD_1cf58c0b-ece7-4f71-8081-fdee5b6fd06d?wsid=/subscriptions/1db33695-8135-4616-9bb4-9574b401d454/resourcegroups/azuremlsdk-rg01/workspaces/azuremlsdk-ws01&tid=6ad91895-de06-485e-bc51-fce126cc8530

Streaming azureml-logs/hyperdrive.txt

[2022-12-11T23:00:48.761796][GENERATOR][INFO]Trying to sample '7' jobs from the hyperparameter space
[2022-12-11T23:00:49.3707487Z][SCHEDULER][INFO]Scheduling job, id='HD_1cf58c0b-ece7-4f71-8081-fdee5b6fd06d_0' 
[2022-12-11T23:00:49.4922248Z][SCHEDULER][INFO]Scheduling job, id='HD_1cf58c0b-ece7-4f71-8081-fdee5b6fd06d_1' 
[2022-12-11T23:00:49.6288801Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_1cf58c0b-ece7-4f71-8081-fdee5b6fd06d_0' 
[2022-12-11T23:00:49.6427093Z][SCHEDULER][INFO]Scheduling job, id='HD_1cf58c0b-ece7-4f71-8081-fdee5b6fd06d_2' 
[2022-12-11T23:00:49.6492199Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_1cf58c0b-ece7-4f71-8081-fdee5b6fd06d_1

ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [40]:
new_run.get_metrics()
#get best run
print(new_run.get_best_run_by_primary_metric())

None


In [34]:
dataset

{
  "source": [
    "('loandatastore', 'Loan_Default.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "a74ed425-18cc-4e50-854a-bb76fa21af43",
    "name": "loansdataset",
    "version": 1,
    "workspace": "Workspace.create(name='azuremlsdk-ws01', subscription_id='1db33695-8135-4616-9bb4-9574b401d454', resource_group='azuremlsdk-rg01')"
  }
}

In [37]:

#import automl
from azureml.train.automl import AutoMLConfig

automlconfig=AutoMLConfig(task='classification',primary_metric='accuracy',label_column_name='Status',training_data=dataset,validation_size=0.25,compute_target=compute,iterations=5,experiment_timeout_minutes=30,featureization='auto')
experiment1=Experiment(workspace,'automlrun')
automl_run=experiment1.submit(automlconfig,snapshot_directory=None)
automl_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automlrun,AutoML_77ae1acb-3af5-4ec5-8249-a670f2657692,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
automlrun,AutoML_77ae1acb-3af5-4ec5-8249-a670f2657692,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of mi

    0   MaxAbsScaler LightGBM                          0:00:19             1.0000    1.0000
    1   MaxAbsScaler XGBoostClassifier                 0:00:24             1.0000    1.0000
    2   MaxAbsScaler ExtremeRandomTrees                0:00:22             1.0000    1.0000
    3    VotingEnsemble                                0:00:41             1.0000    1.0000
    4    StackEnsemble                                 0:00:50             1.0000    1.0000


{'runId': 'AutoML_77ae1acb-3af5-4ec5-8249-a670f2657692',
 'target': 'cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-12-12T01:53:13.678182Z',
 'endTimeUtc': '2022-12-12T02:09:36.638827Z',
 'services': {},
 'properties': {'num_iterations': '5',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0.25',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automlrun","subscription_id":"1db33695-8135-4616-9bb4-9574b401d454","resource_group":"azuremlsdk-rg01","workspace_name":"azuremlsdk-ws01","region":"eastus2","compute_target":"cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":true,"iterations":5,"primary_metric":"accuracy","task_type":"classificatio

In [44]:
#get best run automl
automl_run.get_best_child()
#save the model
best_run, fitted_model=automl_run.get_output()
#save the model


Package:azureml-core, training version:1.47.0, current version:1.44.0
Package:azureml-dataprep, training version:4.5.7, current version:4.0.4
Package:azureml-dataprep-rslex, training version:2.11.4, current version:2.6.3
Package:azureml-dataset-runtime, training version:1.47.0, current version:1.43.0.post2
Package:azureml-interpret, training version:1.47.0, current version:1.44.0
Package:azureml-pipeline-core, training version:1.47.0, current version:1.43.0
Package:azureml-telemetry, training version:1.47.0, current version:1.43.0
Package:azureml-train-automl-client, training version:1.47.0, current version:1.43.0
Package:azureml-train-core, training version:1.47.0, current version:1.43.0
Package:azureml-train-restclients-hyperdrive, training version:1.47.0, current version:1.43.0
Package:azureml-defaults, training version:1.47.0
Package:azureml-inference-server-http, training version:0.7.6
Package:azureml-mlflow, training version:1.47.0
Package:azureml-responsibleai, training version:

In [46]:
#save model onnx

#save model pickle
import pickle
with open('automl_model.pkl','wb') as f:
    pickle.dump(fitted_model,f)
